In [ ]:
import argparse
import pandas as pd
import requests
import pickle
from bs4 import BeautifulSoup # To parse html text
import json, codecs

# PATHES

In [ ]:
TEMPT_file_path = './FILES/TEMPT'
links_cvs_file_path = './FILES/LINKS_TEST.csv'
categories_json_path = './FILES/categories.json'
categories_dict_json_path = './FORMS/categories_dict.json'

# BASIC FUNCTIONS

In [ ]:
def parse_html(url):
    html_content = requests.get(url).content
    return BeautifulSoup(html_content, 'html.parser')

In [ ]:
def read_CSV(filename):
    return pd.DataFrame.from_csv(filename)

# Create 'categories.json' and  'categories_dict.json'

In [ ]:
post_url = 'https://tuoitre.vn/'
try:
    post_tree = parse_html(post_url)
except:
    print 'Cannot read ' + post_url

categories = {}
categories_dict = {}

for item in post_tree.find(class_="clearfix fl").find_all('a'):
    if(item['title'] != 'Home' and item['title'] != 'Media'):
        
        categories_2 = {}
        
        if(item['href'].find('https') == -1):
            url = 'https://tuoitre.vn' + item['href']
            
            categories_dict[item['href'].replace(".htm", "").replace("/", "")] = item['title'].rstrip().lstrip()
            
            tree = parse_html(url)
            
            for item2 in tree.find(class_="list-bc").find_all('a'):
                if(item2['href'].find('https') == -1):
                    categories_2[item2['title']] = item2['href']
                    categories_dict[item2['href'].split("/")[2].replace(".htm", "")] = item2['title'].rstrip().lstrip()

            if(len(categories_2) > 0):
                categories[item['title']] = categories_2
            else:
                categories[item['title']] = item['href']
        else:
            categories[item['title']] = item['href']
        
        
with codecs.open(categories_json_path, 'w', encoding='utf-8') as fp:
    json.dump(categories, fp, ensure_ascii=False)
    
with codecs.open(categories_dict_json_path, 'w', encoding='utf-8') as fp:
    json.dump(categories_dict, fp, ensure_ascii=False)

# create url by time

In [ ]:
from datetime import datetime
from datetime import timedelta

def list_all_day_in_month(url, year, day = 1 ,month = 1):
    url_list_in_moth = []
    front_URL = url + "/xem-theo-ngay/"
    str_date = str(year) + '-' + str(month) + '-' + str(day)
    date_object = datetime.strptime(str_date, '%Y-%m-%d')
    while(int(date_object.month) == month):
        if (date_object < datetime.now()):
            url_list_in_moth.append(front_URL + date_object.strftime('%#d-%#m-%Y') + '.html')
            date_object += timedelta(days=1)
        else:
            break
    return url_list_in_moth

In [ ]:
first_year = 2008
last_year = 2019

with codecs.open(categories_json_path, 'r', encoding='utf-8') as fp:
    categories = json.load(fp)
    url_links = []

for title1 in categories:
    
    back = categories[title1]
    
    if (isinstance(back, basestring)):
        href = back
        if(href.find('https') == -1):
            url = 'https://tuoitre.vn' + href.replace(".htm",'')
            for year in range(first_year, last_year):
                    for month in range(1,13):
                        url_links.extend(list_all_day_in_month(url, year = year, month = month))
    else:
        categories_lv2 = back
        href_main = ""
        for title2 in categories_lv2:
            
            if(categories_lv2[title2].find('https') == -1):
                href_main = categories_lv2[title2].split('/')[1]
                href2 = categories_lv2[title2].split('/')[2]
                
                url2 = 'https://tuoitre.vn/' + href2.replace(".htm",'')
                for year in range(first_year, last_year):
                        for month in range(1,13):
                            url_links.extend(list_all_day_in_month(url2, year = year, month = month))
                        
        if(href_main != ""):
            url_main = 'https://tuoitre.vn/' + href_main
            for year in range(first_year, last_year):
                for month in range(1,13):
                    url_links.extend(list_all_day_in_month(url_main, year = year, month = month))

with codecs.open(TEMPT_file_path, 'wb') as fp:
    pickle.dump(url_links, fp)

# get links

In [ ]:
def get_links(url, date): # them do chiu dung
    url_list = []
    

    while True:
        tree = parse_html(url)
        list_news_content = tree.find(class_="list-news-content pdt30")

        if(list_news_content != None):
            break
            
    url_dict = {}

    for item in list_news_content.find_all(class_="block-right-info fr"):
        url_dict['url'] = 'https://tuoitre.vn' + item.find('a')['href']
        url_dict['category'] = url.split('/')[3]
        url_dict['day'] = date[0]
        url_dict['month'] = date[1]
        url_dict['year'] = date[2]
        
        url_list.append(url_dict)
    return url_list

In [ ]:
def get_page_links(link):
    links_tuple = []
    
    date = link.split('/')[5].replace('.html', '').split('-')

    html_tree = parse_html(link)
    mgr = html_tree.find(class_="page_right mgr15")

    if mgr != None:
        mgr15 = mgr.find_all('a')
        for item in mgr15[:len(mgr15) - 1]:
            links_tuple.extend(get_links('https://tuoitre.vn' + item['href'], date))
    else:
        links_tuple.extend(get_links(link, date))
    return links_tuple

In [ ]:
from time import sleep

i = 0

try:
    df  = read_CSV(links_cvs_file_path)
except:
    df =pd.DataFrame(columns=['url','category','day','month','year'])
    df.to_csv(links_cvs_file_path)

with codecs.open(TEMPT_file_path, 'rb') as fp:
    url_links = pickle.load(fp)

tmp = []

for link in url_links:
    
    if(i%100 == 0 and i > 0):
        with codecs.open(TEMPT_file_path, 'wb') as fp:
            pickle.dump(url_links, fp)
        df = df.append(pd.DataFrame(tmp), ignore_index=True)
        df.to_csv(links_cvs_file_path)
        tmp = []
        sleep(10)
    
    tmp.extend(get_page_links(url_links[i]))
    
    url_links.remove(url_links[i])
    

    i += 1
    print "\r%d %d" %(len(url_links),len(tmp)),